<a href="https://colab.research.google.com/github/luanakwon/ML_Pytorch_pre2022/blob/main/train_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Omok  
---
Simple omok AI using CNN. (probably a little modification of some tuto) 

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
import os
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [ ]:
h, w = 20,20

base_path = 'drive/MyDrive/omok/dataset/*.npz'

file_list = glob(base_path)

x_data, y_data = [],[]
for index, file_path in enumerate(file_list):
  if index < 3000:
    continue
  elif index < 3500:
    print("\r%d" % index, end='')
  else:
    break
  data = np.load(file_path)
  x_data.extend(data['inputs'])
  y_data.extend(data['outputs'])
  

x_val = np.array(x_data, np.float32).reshape((-1,h,w,1))
y_val = np.array(y_data, np.float32).reshape((-1,h*w))

#x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)

del x_data, y_data

#print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


3499(277356, 20, 20, 1) (277356, 400)


In [ ]:
model = models.Sequential([
  layers.Conv2D(64,7,activation='relu', padding='same', input_shape=(h,w,1)),
  layers.Conv2D(128,7,activation='relu',padding='same'),
  layers.Conv2D(256,7,activation='relu',padding='same'), 
  layers.Conv2D(128,7,activation='relu',padding='same'),
  layers.Conv2D(64,7,activation='relu',padding='same'),   
  layers.Conv2D(1,1,activation=None,padding='same'), 
  layers.Reshape((h*w,)),
  layers.Activation('sigmoid') 
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 20, 64)        3200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 128)       401536    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 256)       1605888   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       1605760   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 20, 20, 64)        401472    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 1)         65        
_________________________________________________________________
reshape (Reshape)            (None, 400)               0

In [ ]:
checkpoint_path = 'drive/MyDrive/omok/checkpoints/best.h5'

#model.load_weights(checkpoint_path)

model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    callbacks=[ModelCheckpoint(checkpoint_path,monitor='val_acc',save_best_only=True,verbose=1,mode='auto'),
      ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
    ],
    validation_data=(x_val, y_val),
    use_multiprocessing=True,
    workers=16
)

Epoch 1/10
4567/4567 [==============================] - 598s 129ms/step - loss: 0.0162 - acc: 0.2051 - val_loss: 0.0072 - val_acc: 0.4746

Epoch 00001: val_acc improved from -inf to 0.47458, saving model to drive/MyDrive/omok/checkpoints/best.h5
Epoch 2/10
4567/4567 [==============================] - 606s 133ms/step - loss: 0.0070 - acc: 0.4857 - val_loss: 0.0065 - val_acc: 0.5052

Epoch 00002: val_acc improved from 0.47458 to 0.50516, saving model to drive/MyDrive/omok/checkpoints/best.h5
Epoch 3/10
4567/4567 [==============================] - 607s 133ms/step - loss: 0.0063 - acc: 0.5168 - val_loss: 0.0061 - val_acc: 0.5212

Epoch 00003: val_acc improved from 0.50516 to 0.52121, saving model to drive/MyDrive/omok/checkpoints/best.h5
Epoch 4/10
4567/4567 [==============================] - 612s 134ms/step - loss: 0.0058 - acc: 0.5477 - val_loss: 0.0060 - val_acc: 0.5344

Epoch 00004: val_acc improved from 0.52121 to 0.53442, saving model to drive/MyDrive/omok/checkpoints/best.h5
Epoch 5

In [ ]:

checkpoint_path = 'drive/MyDrive/omok/checkpoints/best.h5'
model.load_weights(checkpoint_path)
model.evaluate(x_val,y_val,batch_size=50,workers=10)

  36/5548 [..............................] - ETA: 3:55:02 - loss: 0.0042 - acc: 0.6842

KeyboardInterrupt: ignored

In [ ]:
from IPython.display import clear_output

class Game:
  def __init__(self, model):
    self.board = np.zeros((20,20),dtype=np.int8)
    self.model = model

  def usrTurn(self):
    while True:
      print("your turn : x y")
      x, y = map(int, input().split())
      if self.board[y,x] == 0:
        self.board[y,x] = -1
        clear_output()
        self.printBoard()
        print("usr placed black at %d,%d"%(x,y))
        break
      else:
        clear_output()
        print("wrong position")

  def aiTurn(self):
    input = np.expand_dims(self.board, axis = (0,-1)).astype(np.float32)

    output = self.model.predict(input).squeeze()
    output = output.reshape((h,w))
    oy, ox = np.unravel_index(np.argmax(output), output.shape)
    if self.board[oy,ox] == 0:
      self.board[oy,ox] = 1
      clear_output()
      self.printBoard()
      print("AI placed white at %d,%d"%(ox,oy))
    else:
      clear_output()
      print("AI error")

  def game_over(self):
    
    for p in [-1,1]:
      for i, row in enumerate(self.board):
        for j, e in enumerate(row):
          if e == p:
            if j < 16 and np.all(self.board[i,j:j+5] == p):
              return p
            if i < 16 and np.all(self.board[i:i+5,j] == p):
              return p
            if i<16 and j<16 and np.all(np.diagonal(self.board[i:i+5,j:j+5])==p):
              return p
    return 0

  def gamestart(self):
    while True:
      self.usrTurn()
      p = self.game_over()
      if p != 0:
        print("player {} wins".format(p))
        break
      self.aiTurn()
      p = self.game_over()
      if p != 0:
        print("player %d wins"%(p))
        break

  def printBoard(self):
    num = '   0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9'
    print(num)
    for i, row in enumerate(self.board):
      print("%2d "%i, end='')
      for el in row:
        if el == 0:
          print('.', end=' ')
        elif el == 1:
          print('0', end=' ')
        else:
          print('@', end=' ')
      print()
    

game = Game(model)
game.gamestart()

   0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9
 0 . . . . . . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . . . . . . . . 
 7 . . . . . . . 0 . . . . . . . . . . . . 
 8 . . . . . . . . @ . . 0 . . . . . . . . 
 9 . . . . . . . @ . @ 0 @ . . . . . . . . 
10 . . . . @ 0 0 0 0 0 @ @ . . . . . . . . 
11 . . . . . . . . @ 0 . @ . . . . . . . . 
12 . . . . . . . . . @ . . 0 . . . . . . . 
13 . . . . . . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . . . . . . 
15 . . . . . . . . . . . . . . . . . . . . 
16 . . . . . . . . . . . . . . . . . . . . 
17 . . . . . . . . . . . . . . . . . . . . 
18 . . . . . . . . . . . . . . . . . . . . 
19 . . . . . . . . . . . . . . . . . . . . 
AI placed white at 5,10
player 1 wins
